##Configurações e importação de bibliotecas 💻

In [ ]:
# Configuração do pyspark

!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz

!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install py4j

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

!pip install -q findspark
import findspark
findspark.init()

!pip install pyspark


Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [910 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,118 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,552 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/

In [ ]:
# Bibliotecas

import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
#from google.colab import drive
from google.colab import files

In [ ]:
spark = SparkSession.builder\
  .master("local")\
  .appName("Datathon")\
  .getOrCreate()

##Base de dados 🎲

In [ ]:
# Base de dados

df_pandas = pd.read_csv("https://raw.githubusercontent.com/Trabalhinho/FIAP_Tech_Challenge/main/Bimestre_5/datathon_base_antiga/PEDE_PASSOS_DATASET_FIAP.csv", sep = ";")

df_csv = spark.createDataFrame(df_pandas)

In [ ]:
# Schema

df_csv.printSchema()

root
 |-- INSTITUICAO_ENSINO_ALUNO_2020: string (nullable = true)
 |-- NOME: string (nullable = true)
 |-- IDADE_ALUNO_2020: string (nullable = true)
 |-- ANOS_PM_2020: string (nullable = true)
 |-- FASE_TURMA_2020: string (nullable = true)
 |-- PONTO_VIRADA_2020: string (nullable = true)
 |-- INDE_2020: string (nullable = true)
 |-- INDE_CONCEITO_2020: string (nullable = true)
 |-- PEDRA_2020: string (nullable = true)
 |-- DESTAQUE_IEG_2020: string (nullable = true)
 |-- DESTAQUE_IDA_2020: string (nullable = true)
 |-- DESTAQUE_IPV_2020: string (nullable = true)
 |-- IAA_2020: string (nullable = true)
 |-- IEG_2020: string (nullable = true)
 |-- IPS_2020: string (nullable = true)
 |-- IDA_2020: string (nullable = true)
 |-- IPP_2020: string (nullable = true)
 |-- IPV_2020: string (nullable = true)
 |-- IAN_2020: string (nullable = true)
 |-- FASE_2021: double (nullable = true)
 |-- TURMA_2021: string (nullable = true)
 |-- INSTITUICAO_ENSINO_ALUNO_2021: string (nullable = true)
 |-- S

In [ ]:
df_csv.show()

+-----------------------------+--------+----------------+------------+---------------+-----------------+------------------+------------------+----------+--------------------+--------------------+--------------------+-----------------+--------+--------+-----------------+-----------------+-----------------+--------+---------+----------+-----------------------------+----------------------------+----------+---------+--------+--------+--------+--------+--------+--------------------+--------------------+--------------------+--------------------+-----------------+--------+--------+--------------------+--------------+---------+----------+-----------------+-------------+-----------------+-------+-------+-------+----------+--------------------+--------------------+--------------------+-----------------+-----------------+--------+-----------------+-----------------+-----------------+-----------------+-------------+-----------------+--------------------+--------------------+--------------------+--

## Tratamentos Iniciais  ⛏

* **Adequação de Layout:** a base está aberta em colunas diferentes para cada ano. Ela será transformada em uma base tabular.


* **Tratamento de nulos e erros:** Alguns valores não fazem sentido e precisam ser removidos:
1. ❌ Aluno 1259
2. ❌ Valor "#NULO" na coluna de INDE


In [ ]:
spark = SparkSession.builder.getOrCreate()

df_csv.createOrReplaceGlobalTempView("base_passos_magicos")

In [ ]:
# PENDENTE: Tratar a idade do Aluno
# tratar o valor #NULO na Coluna inde

base = spark.sql('''SELECT
                          2020 AS ANO,
                          NOME,
                          INSTITUICAO_ENSINO_ALUNO_2020 AS INSTITUICAO_ENSINO_ALUNO,
                          IDADE_ALUNO_2020 AS IDADE_ALUNO,
                          ANOS_PM_2020 AS ANOS_PM,
                          SUBSTRING(FASE_TURMA_2020, 1, 1) AS FASE,
                          SUBSTRING(FASE_TURMA_2020, 2, 1) AS TURMA,
                          PONTO_VIRADA_2020 AS PONTO_VIRADA,
                          INDE_2020 AS INDE,
                          PEDRA_2020 AS PEDRA,
                          DESTAQUE_IEG_2020 AS DESTAQUE_IEG,
                          DESTAQUE_IDA_2020 AS DESTAQUE_IDA,
                          DESTAQUE_IPV_2020 AS DESTAQUE_IPV,
                          IAA_2020 AS IAA,
                          IEG_2020 AS IEG,
                          IPS_2020 AS IPS,
                          IDA_2020 AS IDA,
                          IPP_2020 AS IPP,
                          IPV_2020 AS IPV,
                          IAN_2020 AS IAN,
                          '' AS SINALIZADOR_INGRESSANTE,
                          REC_EQUIPE_1_2021 AS RECOMENDACAO_EQUIPE_1,
                          REC_EQUIPE_2_2021 AS RECOMENDACAO_EQUIPE_2,
                          REC_EQUIPE_3_2021 AS RECOMENDACAO_EQUIPE_3,
                          REC_EQUIPE_4_2021 AS RECOMENDACAO_EQUIPE_4,
                          '' AS NIVEL_IDEAL,
                          CAST(NULL AS FLOAT) AS DEFASAGEM,
                          '' AS BOLSISTA,
                          CAST(NULL AS FLOAT) AS CG,
                          CAST(NULL AS FLOAT) AS CF,
                          CAST(NULL AS FLOAT) AS CT,
                          CAST(NULL AS FLOAT) AS NOTA_PORT,
                          CAST(NULL AS FLOAT) AS NOTA_MAT,
                          CAST(NULL AS FLOAT) NOTA_ING,
                          CAST(NULL AS INTEGER) AS QTD_AVALIACOES,
                          '' AS INDICADO_BOLSA
                        FROM global_temp.base_passos_magicos
                        WHERE NOME != "ALUNO-1259"
                          AND INDE_2020 IS NOT NULL
                          AND INDE_2020 != "NaN"


                        UNION ALL


                        SELECT
                          2021 AS ANO,
                          NOME,
                          INSTITUICAO_ENSINO_ALUNO_2021 AS INSTITUICAO_ENSINO_ALUNO,
                          NULL AS IDADE_ALUNO,
                          NULL AS ANOS_PM,
                          FASE_2021 AS FASE,
                          TURMA_2021 AS TURMA,
                          PONTO_VIRADA_2021 AS PONTO_VIRADA,
                          INDE_2021 AS INDE,
                          PEDRA_2021 AS PEDRA,
                          NULL AS DESTAQUE_IEG,
                          NULL AS DESTAQUE_IDA,
                          NULL AS DESTAQUE_IPV,
                          IAA_2021 AS IAA,
                          IEG_2021 AS IEG,
                          IPS_2021 AS IPS,
                          IDA_2021 AS IDA,
                          IPP_2021 AS IPP,
                          IPV_2021 AS IPV,
                          IAN_2021 AS IAN,
                          SINALIZADOR_INGRESSANTE_2021 AS SINALIZADOR_INGRESSANTE,
                          REC_EQUIPE_1_2021 AS RECOMENDACAO_EQUIPE_1,
                          REC_EQUIPE_2_2021 AS RECOMENDACAO_EQUIPE_2,
                          REC_EQUIPE_3_2021 AS RECOMENDACAO_EQUIPE_3,
                          REC_EQUIPE_4_2021 AS RECOMENDACAO_EQUIPE_4,
                          NIVEL_IDEAL_2021 AS NIVEL_IDEAL,
                          DEFASAGEM_2021 AS DEFASAGEM,
                          NULL AS BOLSISTA,
                          NULL AS CG,
                          NULL AS CF,
                          NULL AS CT,
                          NULL AS NOTA_PORT,
                          NULL AS NOTA_MAT,
                          NULL AS NOTA_ING,
                          NULL AS QTD_AVALIACOES,
                          NULL AS INDICADO_BOLSA
                        FROM global_temp.base_passos_magicos
                        WHERE NOME != "ALUNO-1259"
                          AND INDE_2021 IS NOT NULL
                          AND INDE_2021 != "NaN"


                        UNION ALL


                        SELECT
                          2022 AS ANO,
                          NOME,
                          NULL AS INSTITUICAO_ENSINO_ALUNO,
                          NULL AS IDADE_ALUNO,
                          2022 - ANO_INGRESSO_2022 AS ANOS_PM,
                          FASE_2022 AS FASE,
                          TURMA_2022 AS TURMA,
                          PONTO_VIRADA_2022 AS PONTO_VIRADA,
                          INDE_2022 AS INDE,
                          PEDRA_2022 AS PEDRA,
                          DESTAQUE_IEG_2022 AS DESTAQUE_IEG,
                          DESTAQUE_IDA_2022 AS DESTAQUE_IDA,
                          DESTAQUE_IPV_2022 AS DESTAQUE_IPV,
                          IAA_2022 AS IAA,
                          IEG_2022 AS IEG,
                          IPS_2022 AS IPS,
                          IDA_2022 AS IDA,
                          IPP_2022 AS IPP,
                          IPV_2022 AS IPV,
                          IAN_2022 AS IAN,
                          NULL AS SINALIZADOR_INGRESSANTE,
                          REC_AVA_1_2022 AS RECOMENDACAO_EQUIPE_1,
                          REC_AVA_2_2022 AS RECOMENDACAO_EQUIPE_2,
                          REC_AVA_3_2022 AS RECOMENDACAO_EQUIPE_3,
                          REC_AVA_4_2022 AS RECOMENDACAO_EQUIPE_4,
                          NIVEL_IDEAL_2022 AS NIVEL_IDEAL,
                          NULL AS DEFASAGEM,
                          BOLSISTA_2022 AS BOLSISTA,
                          CG_2022 AS CG,
                          CF_2022 AS CF,
                          CT_2022 AS CT,
                          NOTA_PORT_2022 AS NOTA_PORT,
                          NOTA_MAT_2022  AS NOTA_MAT,
                          NOTA_ING_2022  AS NOTA_ING,
                          QTD_AVAL_2022 AS QTD_AVALIACOES,
                          INDICADO_BOLSA_2022 AS INDICADO_BOLSA
                        FROM global_temp.base_passos_magicos
                        WHERE NOME != "ALUNO-1259"
                          AND INDE_2022 IS NOT NULL
                          AND INDE_2022 != "NaN"''')
base.show()


+----+--------+------------------------+-----------+-------+----+-----+------------+------------------+--------+--------------------+--------------------+--------------------+------------------+---+-----+-----------------+-----------------+-----------------+---+-----------------------+---------------------+---------------------+---------------------+---------------------+-----------+---------+--------+----+----+----+---------+--------+--------+--------------+--------------+
| ANO|    NOME|INSTITUICAO_ENSINO_ALUNO|IDADE_ALUNO|ANOS_PM|FASE|TURMA|PONTO_VIRADA|              INDE|   PEDRA|        DESTAQUE_IEG|        DESTAQUE_IDA|        DESTAQUE_IPV|               IAA|IEG|  IPS|              IDA|              IPP|              IPV|IAN|SINALIZADOR_INGRESSANTE|RECOMENDACAO_EQUIPE_1|RECOMENDACAO_EQUIPE_2|RECOMENDACAO_EQUIPE_3|RECOMENDACAO_EQUIPE_4|NIVEL_IDEAL|DEFASAGEM|BOLSISTA|  CG|  CF|  CT|NOTA_PORT|NOTA_MAT|NOTA_ING|QTD_AVALIACOES|INDICADO_BOLSA|
+----+--------+------------------------+--

In [ ]:
base_tratada = base.select("*").toPandas()

In [ ]:
base_tratada.head(10)

,ANO,NOME,INSTITUICAO_ENSINO_ALUNO,IDADE_ALUNO,ANOS_PM,FASE,TURMA,PONTO_VIRADA,INDE,PEDRA,...,DEFASAGEM,BOLSISTA,CG,CF,CT,NOTA_PORT,NOTA_MAT,NOTA_ING,QTD_AVALIACOES,INDICADO_BOLSA
0,2020,ALUNO-1,Escola Pública,11,2,2,H,Não,7.883752,Ametista,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,2020,ALUNO-3,Escola Pública,12,2,3,H,Não,7.856389666666666,Ametista,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,2020,ALUNO-4,Escola Pública,10,2,1,D,Não,5.076252,Quartzo,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,2020,ALUNO-5,Escola Pública,10,1,2,M,Não,8.077085,Ametista,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,2020,ALUNO-8,Rede Decisão/União,14,3,4,L,Sim,8.381390666666666,Ametista,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
5,2020,ALUNO-10,Escola Pública,13,1,3,B,Não,5.1618745,Quartzo,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
6,2020,ALUNO-11,Escola Pública,8,0,0,A,Sim,9.712504000000001,Topázio,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
7,2020,ALUNO-12,Rede Decisão/União,14,3,3,G,Não,5.838887333333334,Quartzo,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
8,2020,ALUNO-14,Escola Pública,11,2,2,M,Não,6.434585,Ágata,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
9,2020,ALUNO-15,Escola Pública,12,1,2,D,Não,5.162502,Quartzo,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [ ]:
len(base_tratada)

2275

In [ ]:
base_tratada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2275 entries, 0 to 2274
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ANO                       2275 non-null   int32  
 1   NOME                      2275 non-null   object 
 2   INSTITUICAO_ENSINO_ALUNO  1413 non-null   object 
 3   IDADE_ALUNO               727 non-null    object 
 4   ANOS_PM                   1589 non-null   object 
 5   FASE                      2275 non-null   object 
 6   TURMA                     2275 non-null   object 
 7   PONTO_VIRADA              2275 non-null   object 
 8   INDE                      2275 non-null   object 
 9   PEDRA                     2275 non-null   object 
 10  DESTAQUE_IEG              1589 non-null   object 
 11  DESTAQUE_IDA              1589 non-null   object 
 12  DESTAQUE_IPV              1589 non-null   object 
 13  IAA                       2275 non-null   object 
 14  IEG     

In [ ]:
#  Tratar a coluna ANOS PM

In [ ]:
# Tratar tipos de dados

base_tratada['IDADE_ALUNO'] = base_tratada['IDADE_ALUNO'].astype(float)

base_tratada[['IAA', 'IEG', 'IPS', 'IDA', 'IPP', 'IPV', 'IAN']] = base_tratada[['IAA', 'IEG', 'IPS', 'IDA', 'IPP', 'IPV', 'IAN']].astype(float)

In [ ]:
base_tratada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2275 entries, 0 to 2274
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ANO                       2275 non-null   int32  
 1   NOME                      2275 non-null   object 
 2   INSTITUICAO_ENSINO_ALUNO  1413 non-null   object 
 3   IDADE_ALUNO               727 non-null    float64
 4   ANOS_PM                   1589 non-null   object 
 5   FASE                      2275 non-null   object 
 6   TURMA                     2275 non-null   object 
 7   PONTO_VIRADA              2275 non-null   object 
 8   INDE                      2275 non-null   object 
 9   PEDRA                     2275 non-null   object 
 10  DESTAQUE_IEG              1589 non-null   object 
 11  DESTAQUE_IDA              1589 non-null   object 
 12  DESTAQUE_IPV              1589 non-null   object 
 13  IAA                       2275 non-null   float64
 14  IEG     

### Análise exploratória 📈

In [ ]:
base_tratada[['IAA', 'IEG', 'IPS', 'IDA', 'IPP', 'IPV', 'IAN', 'INDE']].describe().T

# o indicador IAN e o unico que nao tem valres zerados

,count,mean,std,min,25%,50%,75%,max
IAA,2275.0,8.263841,2.012095,0.0,7.916670,8.750010,9.500000,10.00002
IEG,2275.0,7.497884,2.205902,0.0,6.300000,8.100000,9.200000,10.00000
IPS,2275.0,6.831462,1.296417,0.0,6.250000,7.500000,7.500000,10.00000
IDA,2275.0,5.956667,2.432544,0.0,4.500000,6.333333,7.666667,10.00000
IPP,2275.0,6.930016,1.542491,0.0,6.250000,7.300000,7.916667,10.00000
IPV,2275.0,7.294262,1.505309,0.0,6.763885,7.500000,8.166670,10.00001
IAN,2275.0,6.889011,2.506651,2.5,5.000000,5.000000,10.000000,10.00000


In [ ]:
# Ver matriz de correlação dos indicadores e os outliers

In [ ]:
base_tratada[['DEFASAGEM']].describe().T


,count,mean,std,min,25%,50%,75%,max
DEFASAGEM,686.0,-0.752187,0.792181,-4.0,-1.0,-1.0,0.0,3.0


In [ ]:
# Quantidade de alunos por ano

base_tratada['ANO'].value_counts()

ANO
2022    862
2020    727
2021    686
Name: count, dtype: int64

In [ ]:
# Quantidade de alunos por escola: A maior parte das escolas não tem o nome citado e são resumidas em "Escola Pública
base_tratada['INSTITUICAO_ENSINO_ALUNO'].value_counts()

INSTITUICAO_ENSINO_ALUNO
Escola Pública          1155
Rede Decisão             127
Rede Decisão/União       101
FIAP                      11
UNISA                     10
Estácio                    3
Einstein                   2
Escola João Paulo II       2
Escola JP II               2
Name: count, dtype: int64

In [ ]:
# Quantidade de alunos por idade: Tratar para aparecer como número inteiro

base_tratada['IDADE_ALUNO'].value_counts()

IDADE_ALUNO
10.0    107
11.0     93
12.0     86
13.0     80
14.0     72
9.0      59
16.0     54
15.0     50
17.0     45
8.0      43
18.0     21
7.0       9
19.0      7
20.0      1
Name: count, dtype: int64

In [ ]:
# Quantidade de alunos por anos na passos mágicos:
# Tratar:
# - se o aluno ingressou em 2020 e tem registro nos próximos anos, somar + 1
# - se ingressou em 2021 somar 1
# - se ingressou em 2021 e tem registro em 2022, somar +1
# - se ingressou em 2022, somar 1

#### Analisar as colunas ANO_INGRESSO_2022 e SINALIZADOR_INGRESSANTE_2021

base_tratada['ANOS_PM'].value_counts()

ANOS_PM
0.0    398
1      305
2      156
1.0    141
3.0    139
0      134
3       78
4.0     67
4       54
2.0     48
5.0     40
6.0     29
Name: count, dtype: int64

In [ ]:
# Quantidade de alunos por fase: Tratar para ter apenas números inteiros

base_tratada['FASE'].value_counts()

FASE
1.0    328
2.0    317
0.0    312
3.0    263
1      172
2      155
4.0    135
3      122
5.0    110
0       82
4       55
5       54
7.0     42
6.0     41
7       33
6       30
8       24
Name: count, dtype: int64

In [ ]:
base_tratada.columns

Index(['ANO', 'NOME', 'INSTITUICAO_ENSINO_ALUNO', 'IDADE_ALUNO', 'ANOS_PM',
       'FASE', 'TURMA', 'PONTO_VIRADA', 'INDE', 'PEDRA', 'DESTAQUE_IEG',
       'DESTAQUE_IDA', 'DESTAQUE_IPV', 'IAA', 'IEG', 'IPS', 'IDA', 'IPP',
       'IPV', 'IAN', 'SINALIZADOR_INGRESSANTE', 'RECOMENDACAO_EQUIPE_1',
       'RECOMENDACAO_EQUIPE_2', 'RECOMENDACAO_EQUIPE_3',
       'RECOMENDACAO_EQUIPE_4', 'NIVEL_IDEAL', 'DEFASAGEM', 'BOLSISTA', 'CG',
       'CF', 'CT', 'NOTA_PORT', 'NOTA_MAT', 'NOTA_ING', 'QTD_AVALIACOES',
       'INDICADO_BOLSA'],
      dtype='object')

In [ ]:
# Excluir colunas que não serão usadas.

In [ ]:
base_tratada_colunas = base_tratada[['ANO', 'NOME', 'ANOS_PM', 'FASE', 'TURMA', 'PONTO_VIRADA', 'INDE', 'PEDRA', 'IAA', 'IEG', 'IPS', 'IDA', 'IPP', 'IPV', 'IAN']]

In [ ]:
base_tratada_colunas.head(10)

,ANO,NOME,ANOS_PM,FASE,TURMA,PONTO_VIRADA,INDE,PEDRA,IAA,IEG,IPS,IDA,IPP,IPV,IAN
0,2020,ALUNO-1,2,2,H,Não,7.883752,Ametista,8.50002,8.7,7.500,7.000000,5.93750,7.750000,10.0
1,2020,ALUNO-3,2,3,H,Não,7.856389666666666,Ametista,7.91667,8.9,7.500,5.500000,8.12500,8.111113,10.0
2,2020,ALUNO-4,2,1,D,Não,5.076252,Quartzo,8.00002,4.1,6.875,0.000000,7.18750,7.750000,5.0
3,2020,ALUNO-5,1,2,M,Não,8.077085,Ametista,7.50002,8.0,7.500,7.500000,8.43750,8.166665,10.0
4,2020,ALUNO-8,3,4,L,Sim,8.381390666666666,Ametista,8.33334,9.9,4.375,7.333333,8.75000,8.944450,10.0
5,2020,ALUNO-10,1,3,B,Não,5.1618745,Quartzo,5.83333,5.2,3.750,0.000000,7.96875,6.833333,10.0
6,2020,ALUNO-11,0,0,A,Sim,9.712504000000001,Topázio,9.00002,10.0,8.125,10.000000,10.00000,10.000010,10.0
7,2020,ALUNO-12,3,3,G,Não,5.838887333333334,Quartzo,6.25000,6.0,5.625,6.666667,5.62500,5.277770,5.0
8,2020,ALUNO-14,2,2,M,Não,6.434585,Ágata,8.00002,7.6,6.875,0.000000,8.43750,7.916665,10.0
9,2020,ALUNO-15,1,2,D,Não,5.162502,Quartzo,8.50002,4.0,7.500,0.000000,8.12500,7.250000,5.0


In [ ]:
base_tratada_colunas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2275 entries, 0 to 2274
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ANO           2275 non-null   int32  
 1   NOME          2275 non-null   object 
 2   ANOS_PM       1589 non-null   object 
 3   FASE          2275 non-null   object 
 4   TURMA         2275 non-null   object 
 5   PONTO_VIRADA  2275 non-null   object 
 6   INDE          2275 non-null   object 
 7   PEDRA         2275 non-null   object 
 8   IAA           2275 non-null   float64
 9   IEG           2275 non-null   float64
 10  IPS           2275 non-null   float64
 11  IDA           2275 non-null   float64
 12  IPP           2275 non-null   float64
 13  IPV           2275 non-null   float64
 14  IAN           2275 non-null   float64
dtypes: float64(7), int32(1), object(7)
memory usage: 257.8+ KB


In [ ]:
base_tratada_colunas.to_excel("tbl_passos_magicos.xlsx")

In [ ]:
base_tratada_colunas_string = base_tratada[['ANO', 'NOME', 'ANOS_PM', 'FASE', 'TURMA', 'PONTO_VIRADA', 'INDE', 'PEDRA', 'IAA', 'IEG', 'IPS', 'IDA', 'IPP', 'IPV', 'IAN']]

In [ ]:
base_tratada_colunas_string['INDE'] = base_tratada_colunas_string['INDE'].astype(str)
base_tratada_colunas_string['PEDRA'] = base_tratada_colunas_string['PEDRA'].astype(str)
base_tratada_colunas_string['IAA'] = base_tratada_colunas_string['IAA'].astype(str)
base_tratada_colunas_string['IEG'] = base_tratada_colunas_string['IEG'].astype(str)
base_tratada_colunas_string['IPS'] = base_tratada_colunas_string['IPS'].astype(str)
base_tratada_colunas_string['IDA'] = base_tratada_colunas_string['IDA'].astype(str)
base_tratada_colunas_string['IPP'] = base_tratada_colunas_string['IPP'].astype(str)
base_tratada_colunas_string['IPV'] = base_tratada_colunas_string['IPV'].astype(str)
base_tratada_colunas_string['IAN'] = base_tratada_colunas_string['IAN'].astype(str)

<ipython-input-31-29dae5e9cdac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_tratada_colunas_string['INDE'] = base_tratada_colunas_string['INDE'].astype(str)
<ipython-input-31-29dae5e9cdac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_tratada_colunas_string['PEDRA'] = base_tratada_colunas_string['PEDRA'].astype(str)
<ipython-input-31-29dae5e9cdac>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [ ]:
base_tratada_colunas_string.to_excel("tbl_passos_magicos_string.xlsx", index = False)

In [ ]:
print( set(base_tratada_colunas_string['ANOS_PM']))

{'0.0', '3.0', '1', '0', '5.0', '4.0', '6.0', '2.0', '1.0', '3', '2', '4', None}
